In [1]:
initial_path = 'peptide-QML'
# initial_path = '..'

In [2]:
import sys, torch
import torch.nn as nn
import torch.nn.functional as F
sys.path.append(initial_path)

%load_ext autoreload
%autoreload 2
from my_code import helper_classes as c
from my_code import quantum_nodes as q
from my_code import vae.VAE as VAE

In [ ]:
class ScorePredictor(c.Module):
    def __init__(self, latent_dim:int):
        super(ScorePredictor, self).__init__()

        # quantum circuit
        quantum_circuit = q.circuit(
            n_qubits = int(q.np.ceil(q.np.log2(latent_dim))),
            device = "default.qubit.torch",
            device_options = {'shots': None},
            embedding = q.parts.AmplitudeEmbedding,
            # embedding_ansatz = sweep_point['ansatz'],
            block_ansatz = q.parts.Ansatz_11,
            final_ansatz = q.parts.Ansatz_11, 
            measurement = q.parts.Measurement('Z', 1),
            # embedding_n_layers = sweep_point['embedding_n_layers'],
            # different_inputs_per_layer = False,
            block_n_layers = 10,
            # wrapper_qlayer = pw.QLayerEmpty,
        )
        self.quantum_descprition = str(quantum_circuit)

        # layers of the model
        self.quantum_layer = quantum_circuit()
        self.post_quantum = nn.Linear(1, 1)
    
    def forward(self, x):
        x = self.quantum_layer(x)
        x = self.post_quantum(x)
        x = x.squeeze(-1)
        return x
    
    @staticmethod
    def loss_function(SP_out, batch, reduction:str='mean'):
        x, y = batch
        return F.mse_loss(SP_out, y.float(), reduction=reduction)
    
    def save(self, path):
        copy_of_self = c.copy.deepcopy(self).to('cpu')
        copy_of_self.quantum_layer = None
        torch.save(copy_of_self, path)

In [ ]:
name = 'vae_TEST.pickle'
vae_model = VAE.load(initial_path+'/saved/Pickle/VAE-'name)
score_predictor = ScorePredictor(latent_dim=vae_model.latent_dim)

In [ ]:
SP = ScorePredictor(6)